In [1]:
from pytorch_unet import *
from torch import nn
import pandas as pd
from timerit import Timerit
from IPython.display import display, HTML

In [2]:
print(torch.__version__)

1.9.0+cu102


In [3]:
model = UNet_rect_kernels(128,2).eval()

x = torch.randn((2,1,128,128))
pred = model(x)

In [4]:
device_name = " "
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(torch.cuda.current_device())
else:
    device_name = 'cpu'
    
print('Using device:', device_name)

device = " "
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

Using device: GeForce RTX 2060 SUPER


In [5]:
metric_labels = [
    "Frames",
    "# Samples",
    "First PT (s)",
    "Avg PT (s)",
    "Min PT (s)",
    "Max PT (s)",
    "Std. Dev. (s)",
]
    
print('Using device:', device_name)

results_df = pd.DataFrame(columns = metric_labels)
frames = [2, 10, 20, 30, 40, 50]
for l in range(len(frames)):

    model = UNet(128,2)
    if device == 'cuda':
        model = model.to(torch.device('cuda'))
    t = Timerit(num = 100, verbose=0)
    
    model = model.eval()
    
    for timer in t:
        x = torch.randn((frames[l],1,128,128))
        with timer:
            x = x.to(device)
            pred = model(x)
    
    mf_times = t.times
    
    results_df.loc[l] = [
        frames[l],
        100,
        mf_times[0],
        int((np.mean(mf_times)*100000)) / 100000,
        np.min(mf_times),
        np.max(mf_times),
        int((np.std(mf_times)*100000)) / 100000
    ]

print('Pytorch Multi-Frame Batch Prediction Timings')
print('"PT" = Prediction Time')
display(results_df)

Using device: GeForce RTX 2060 SUPER
Pytorch Multi-Frame Batch Prediction Timings
"PT" = Prediction Time


,Frames,# Samples,First PT (s),Avg PT (s),Min PT (s),Max PT (s),Std. Dev. (s)
0,2.0,100.0,0.016310,0.00273,0.002552,0.016310,0.00136
1,10.0,100.0,0.003863,0.00268,0.002648,0.003863,0.00013
2,20.0,100.0,0.003505,0.00281,0.002729,0.003505,0.00013
3,30.0,100.0,0.004076,0.00318,0.002916,0.004076,0.00025
4,40.0,100.0,0.004175,0.00320,0.003012,0.004943,0.00028
5,50.0,100.0,0.004961,0.00340,0.003146,0.004961,0.00021
